# Explore GRS images

## Load python libraries

In [ ]:

import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray  # activate the rio accessor

#import cartopy
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('TkAgg')
import hvplot.xarray

import holoviews as hv
import holoviews.operation.datashader as hd
hv.extension('bokeh')
import datashader as ds
import colorcet as cc
import panel as pn
import panel.widgets as pnw
#import ipywidgets as widgets

import pyproj as ppj
import rasterio
from affine import Affine
from shapely.geometry import box,Point, mapping
from shapely.ops import transform


import grstbx
from grstbx import visual

u = grstbx.utils
opj = os.path.join


## Set Dask local cluster

In [ ]:
import dask.distributed
cluster = dask.distributed.LocalCluster(processes=False)
client = dask.distributed.Client(cluster)
client

## Set images to play with

In [ ]:

satdir = '/sat_data/satellite/sentinel2/L2A/GRS/31TGM'


image='S2*_v14.nc'
files = glob.glob(opj(satdir, image))
files

In [ ]:
files = pn.widgets.FileSelector(satdir,file_pattern='*.nc')
files

## Load and subset image series

In [ ]:
# central coordinate
lon, lat = 6.58, 46.35
# size in meter of the rectangle of interest
width, height = 40000, 18000

ust = grstbx.utils.spatiotemp()
box = ust.wktbox(lon,lat, width=width, height=height, ellps='WGS84')
bbox = gpd.GeoSeries.from_wkt([box]).set_crs(epsg=4326)
# reproject lon lat in xy coordinates
bbox = bbox.to_crs(epsg=32631)

# generate datacube
dc = grstbx.l2grs(files.value)
dc.load(subset=bbox.bounds.values[0])

#bbox = bbox.to_crs(epsg=3857)
#dc.load(reproject=True, subset=bbox.bounds.values[0])


check bounds

In [ ]:
 dc.bbox.to_crs(4326).bounds

## Check metadata

In [ ]:
pn.widgets.DataFrame(pd.DataFrame.from_dict(dc.datacube.metadata.attrs,orient='index'),height=300)

## **Check spectral datacube** (i.e., Remote Sensing Reflectance, R<sub>rs</sub>, sr<sup>-1</sup>)

In [ ]:
dc.Rrs

To quickly check your data visually, you can use the *visual* module of *grstbx*

In [ ]:
visual.image_viewer().Rrs_date(dc.Rrs.Rrs)

## Check flags and masking

In [ ]:
masking_ = grstbx.masking(dc.Rrs)
pn.widgets.DataFrame(masking_.print_info(),height=400)

In [ ]:
#mask_ = masking_.get_mask(negative=True)
#mask_
#hv.Image(hv.Dataset(mask_,  kdims=['x','y']))
Rrs_masked = dc.Rrs.Rrs.where(dc.Rrs.Rrs.isel(wl=1)>0).where(dc.Rrs.Rrs.isel(wl=8)>0)

## Masking pixels from flags

In [ ]:
#p_ = p_.where(p_.mask_ndwi_corr_mask)
#p_.mask_ndwi_corr_mask

## **Fast checking of the RGB images**

In [ ]:
bands=[3,2,1]
Rrs_masked.isel(wl=bands).plot.imshow(col='time', robust=True, figsize=(18, 6))


In [ ]:
bands=[4,2,1]
Rrs_masked.isel(wl=bands).plot.imshow(col='time', robust=True, figsize=(18, 6))

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf 
from colorcet import palette


shaded = []
for name, raster in Rrs_masked.isel(wl=2).groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.kbc)
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 3
imgs

# L2B algorithm to get biogeochemical parameters
## Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:
# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]

ratio = np.log10(Rrs_masked.isel(wl=1)/Rrs_masked.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
chl = 10**(logchl)
chl.name='chl in mg.m-3 from OC2'

Set range of valid values

In [ ]:
chl = chl.where((chl >= 0) & (chl <= 300))
chl.compute()

In [ ]:
visual.image_viewer().param_date(chl)

In [ ]:
raster = chl


img = hv.Image(raster.squeeze().isel(time=2))#,cmap=cc.bgyw)
img

In [ ]:
raster = chl

shaded = []
for name, raster in chl.groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.bgyw)
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 3
imgs

In [ ]:

import datashader.transfer_functions as tf
shaded = []
for name, raster in dc.Rrs.Rrs.isel(time=0).groupby('wl'):
    img = tf.shade(raster,cmap=cc.kbc)
    img.name = '{:.2f}'.format(name)+' nm'
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 3
imgs

In [ ]:
import datashader as ds
ds.utils.export_image(imgs.images, "lake_geneva_blue.png", export_path="../illustration",background=None)




In [ ]:

Rrs_render.hvplot.quadmesh(groupby='wl',
                           geo=True, crs=Rrs_render.rio.crs.to_proj4(),
                           widget_location='bottom',height=500,width=800,clim=(0,0.07))


In [ ]:
cmap = 'RdBu_r'
hv.Layout([hv.Image(Rrs_render.isel(wl=ib),['lon','lat']).relabel(str(Rrs_render.wl.data[ib])+' nm')\
           .opts(cmap=cmap, \
                 width=300, height=300) for ib in range(9)]).redim.range(value=(-.3, .3)).cols(4) #.opts(colorbar=True, )

In [ ]:
image=rgb
edge=regb
cmaps = [cmap for cmap in hv.plotting.list_cmaps() if (cmap.endswith("_r") and cmap.islower())]

cmap = pn.widgets.Select(
    value="binary_r", options=cmaps, name="Color Map"
)

before_img = hv.Image(image, ).apply.opts(
    cmap=cmap, title="Before", active_tools=["box_zoom"], responsive=True
)
after_img = hv.Image(edges, ).apply.opts(
    cmap=cmap, title="After", active_tools=["box_zoom"], responsive=True
)

before = pn.panel(before_img)
after = pn.panel(after_img)

component = pn.Row(before, after, height=2 * height)

In [ ]:
import geoviews as gv
gv_dataset = gv.Dataset(dc.Rrs.Rrs, kdims=['wl','y','x'], vdims=['Raster'] , crs=ccrs.UTM(17))
#hd.regrid(gv_dataset.to(gv.Image, kdims=['x','y']))
image = hd.regrid(gv_dataset.to(gv.Image, kdims=['x','y']))
image

In [ ]:
import geoviews as gv
import geoviews.feature as gf

from geoviews import opts
from cartopy import crs

gv.extension('bokeh', 'matplotlib')

features = gv.Overlay([gf.ocean, gf.land, gf.rivers, gf.lakes, gf.borders, gf.coastline])

gv.output(features, backend='matplotlib', fig='svg', size=300)
